<style>
  @keyframes glow {
    0% { text-shadow: 0 0 10px #1E90FF; }
    50% { text-shadow: 0 0 20px #1E90FF; }
    100% { text-shadow: 0 0 10px #1E90FF; }
  }
  .justify-text {
    text-align: justify;
    margin: 0 auto;  /* Centre le conteneur */
    padding: 0 10px; /* 20px de padding à gauche et à droite */
    max-width: 900px; /* Largeur maximale pour limiter l'étendue du texte */
  }

</style>

<div class="justify-text">
  <h1 style="text-align: center; font-size: 3.5em; color: #1E90FF; border: 2px solid #1E90FF; animation: glow 1s infinite alternate; padding: 10px;">🛠️ Tools Kit 🛠️</h1>

  <p align="center">
    <a href="https://schwallergroup.github.io/practical-programming-in-chemistry/">
      <img src="https://img.shields.io/badge/PPCHEM-3776AB.svg?style=for-the-badge&logo=python&logoColor=white" alt="PPCHEM">
    </a>
  </p>

  <h2>📋 <strong>Project Context & ⚔️ Challenge Faced</strong></h2>
  <hr>
  <p>🚀 Starting this project was challenging because it was one of the first times we were so free. We had to estimate what would be interesting to tackle, what would be technically feasible, and what we were capable of. So, we began by opening a document in <strong>VS Code</strong> and trying out multiple things. The first idea that came to mind was a way to estimate ligand efficiency using natural product scores and other factors. However, we quickly realized that the research topics were too recent and the project risked being <span style="color: #1E90FF;">too ambitious.</span> Since we weren't very clear about our capabilities at that time, the idea of <span style="color: #1E90FF;">creating a kit</span> with different functionalities, some easier to add than others, and that could be improved as desired, emerged.</p>

  <p>🌟 We soon realized that for this project to be engaging, it needed to be attractive and interactive. If the code remained raw in a <strong>Jupyter</strong> cell, it wouldn't be appealing to use. So, the question arose: <span style="color: #1E90FF;">web page or interface? 🌐</span> A web page seemed attractive because if we could host our site, it would be simply on the internet and easily accessible, even for our parents. So, we started testing with <strong>Django</strong>, and after a day spent making a "Hello world" appear, we realized that coding it from scratch would be too extensive. Not knowing about platforms like <strong>WordPress</strong> at that time, our choice shifted to the <span style="color: #1E90FF;">interface.</span> That's when the project really began. At that point, the functionalities were limited to creating a simple graph without options, obtaining the molar mass from a <strong>SMILES</strong>, and generating a SMILES from a molecular formula.</p>

  <p>🎨 The problem that arose here was that we realized a manually coded simple interface was extremely limited in terms of <span style="color: #1E90FF;">aesthetics.</span> So, we discovered <strong>Tkinter Designer</strong> on <strong>GitHub</strong>, a package that allowed us to transition from manually designing an interface on <strong>figma</strong> to <strong>Python</strong> code. Although it was still limited in terms of capabilities, this allowed us to create an interface exactly as we desired. Multiple tests were conducted, and the interface was improved multiple times, but this is where the project took on a whole new dimension. With a significantly improved interface, our aspirations for this project soared to new heights 🚀.</p>

  <p>🔍 We then continued to add functionalities such as linear regression when we learned that the final project had to be on a notebook, so we had to translate our <code>.py</code> into a <code>.ipynb</code>. This wasn't easy because some functionalities <span style="color: #1E90FF;">no longer worked</span>; for example, the graph windows didn't appear, so we had to define new functions, and that's when we decided to add all these functionalities to our graphs.</p>

  <p>💡 One of the main problems later on was the <span style="color: #1E90FF;">limitation imposed by the <strong>Tkinter</strong> window</span>. Many times we thought we could add this or that functionality, and we spent hours and hours trying to implement it until we realized it wasn't within our abilities. At that point, our main functionalities were the graph, linear regression, and molecular structure, but we were <span style="color: #1E90FF;">missing something that would make</span> our project a <span style="color: #1E90FF;">true tools <strong>kit</strong></span>, as its name suggests. After many attempts and efforts, we finally found and implemented the function for <span style="color: #1E90FF;">error propagation.</span></p>

  <p>🛠️ The <span style="color: #1E90FF;">last major problem</span> we encountered was turning this "*simple*" notebook into a <span style="color: #1E90FF;">package.</span> We, who knew nothing about it, had to learn everything about <code>.toml</code>, <code>.gitignore</code>, tests, etc... The tests, in particular, were not easy because it wasn't just a matter of testing a simple addition function; we had to test functions related to interfaces, such as the background color of the graph on a <strong>Tkinter</strong> window.</p>

  <p>🌍 Ultimately, this project taught us a lot about what we might encounter in the world of work: spending weeks and weeks on the same thing, learning to collaborate, being rigorous, facing failure, and much more. The most beautiful outcome we encountered was when we realized that in the future, we would actually use this tool in our daily lives.</p>

  <p>🙏 Thank you for reading, and we hope you <span style="color: #FF007F; text-decoration: underline;">enjoy</span> exploring this project as much as we enjoyed creating it.</p>

  <hr>
</div>




### 📚 Libraries and Packages
In this section, we import various essential libraries for our project. These libraries provide functionalities for graphical user interfaces, chemical computations, and data visualization.

#### 🔧 Tool Libraries
- **tkinter**: Used for creating graphical user interfaces.
- **rdkit**: A toolkit for cheminformatics.
- **sys**: Provides access to system-specific parameters and functions.
- **matplotlib**: A plotting library for creating static, animated, and interactive visualizations.
- **datetime**: Supplies classes for manipulating dates and times.


In [29]:
%matplotlib inline
import tkinter as tk
from tkinter import messagebox, filedialog, Canvas, Entry, Button, PhotoImage
from rdkit.Chem import rdDepictor
import sys
import matplotlib.pyplot as plt
from matplotlib_inline.backend_inline import set_matplotlib_formats
from datetime import datetime


### 🛠️ Configuration Setup

#### 📁 Library Path
We set up the library path to include the directory "../src" as the first entry in the system path. This allows us to import modules from this directory.

#### 🎨 Chemical Drawing Preferences
We configure the RDKit library to prefer coordinate generation during molecule depiction, enhancing the quality of chemical drawings.

#### 📊 Plotting Formats
We specify the formats for matplotlib plots as PNG and PDF using the `set_matplotlib_formats` function from the `matplotlib_inline.backend_inline` module.

#### 📝 Font Family
We set the font family for matplotlib plots to "Times New Roman" for consistent and aesthetically pleasing text rendering.



In [30]:
sys.path.insert(0, "../src")
rdDepictor.SetPreferCoordGen(True)
set_matplotlib_formats('png', 'pdf')
plt.rcParams['font.family'] = 'Times New Roman'

## 🧪 Features

 ### 1. 🧬 Molecule Name
   - Input: Raw formula of the molecule
   - Output: SMILES of the molecule

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed dictum consectetur lectus, vitae tincidunt urna accumsan vel. Nunc in enim nulla. Vestibulum eget lorem nec nisi lobortis feugiat. Nullam lacinia pharetra lorem, vitae hendrerit dui fermentum nec. Sed non felis nec justo posuere interdum. Integer euismod magna nec dui vehicula, id fermentum orci hendrerit. Integer scelerisque nec urna vel lacinia. Vivamus convallis, dui a euismod lacinia, ex enim tincidunt magna, vitae suscipit lacus enim non nisi. Curabitur id justo justo. Nunc accumsan eget arcu at rhoncus. Etiam at sapien vel odio scelerisque fermentum. Sed tincidunt, justo a ultricies malesuada, sem justo consequat tellus, et ultricies elit arcu ac lorem. Ut auctor purus nec libero sodales, quis consectetur eros hendrerit. Vivamus varius semper neque, at tincidunt neque sollicitudin at.
### 2. ⚖️ Molecular Weight
   - Input: SMILES of the molecule
   - Output: Corresponding molar mass in g/mol

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed dictum consectetur lectus, vitae tincidunt urna accumsan vel. Nunc in enim nulla. Vestibulum eget lorem nec nisi lobortis feugiat. Nullam lacinia pharetra lorem, vitae hendrerit dui fermentum nec. Sed non felis nec justo posuere interdum. Integer euismod magna nec dui vehicula, id fermentum orci hendrerit. Integer scelerisque nec urna vel lacinia. Vivamus convallis, dui a euismod lacinia, ex enim tincidunt magna, vitae suscipit lacus enim non nisi. Curabitur id justo justo. Nunc accumsan eget arcu at rhoncus. Etiam at sapien vel odio scelerisque fermentum. Sed tincidunt, justo a ultricies malesuada, sem justo consequat tellus, et ultricies elit arcu ac lorem. Ut auctor purus nec libero sodales, quis consectetur eros hendrerit. Vivamus varius semper neque, at tincidunt neque sollicitudin at.

### 3. 📈 Linear Regression
   - Input: Excel document (imported by pressing the Browse button)
   - Output: Linear regression graph with the R<sup>2</sup> value
        - Options: The graph allows customization of various parameters such as changing scales, axis labels, title, displaying maximum values, enabling gridlines, adjusting line types and colors, modifying grid and label axes, and background color.

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed dictum consectetur lectus, vitae tincidunt urna accumsan vel. Nunc in enim nulla. Vestibulum eget lorem nec nisi lobortis feugiat. Nullam lacinia pharetra lorem, vitae hendrerit dui fermentum nec. Sed non felis nec justo posuere interdum. Integer euismod magna nec dui vehicula, id fermentum orci hendrerit. Integer scelerisque nec urna vel lacinia. Vivamus convallis, dui a euismod lacinia, ex enim tincidunt magna, vitae suscipit lacus enim non nisi. Curabitur id justo justo. Nunc accumsan eget arcu at rhoncus. Etiam at sapien vel odio scelerisque fermentum. Sed tincidunt, justo a ultricies malesuada, sem justo consequat tellus, et ultricies elit arcu ac lorem. Ut auctor purus nec libero sodales, quis consectetur eros hendrerit. Vivamus varius semper neque, at tincidunt neque sollicitudin at.

### 4. 📊 Graph Maker
   - Input: Excel document with x and y values arranged in two columns
   - Output: Graph plotting all values
      - Options: The graph allows customization of various parameters such as changing scales, axis labels, title, displaying maximum values, enabling gridlines, adjusting line types and colors, modifying grid and label axes, and background color.

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed dictum consectetur lectus, vitae tincidunt urna accumsan vel. Nunc in enim nulla. Vestibulum eget lorem nec nisi lobortis feugiat. Nullam lacinia pharetra lorem, vitae hendrerit dui fermentum nec. Sed non felis nec justo posuere interdum. Integer euismod magna nec dui vehicula, id fermentum orci hendrerit. Integer scelerisque nec urna vel lacinia. Vivamus convallis, dui a euismod lacinia, ex enim tincidunt magna, vitae suscipit lacus enim non nisi. Curabitur id justo justo. Nunc accumsan eget arcu at rhoncus. Etiam at sapien vel odio scelerisque fermentum. Sed tincidunt, justo a ultricies malesuada, sem justo consequat tellus, et ultricies elit arcu ac lorem. Ut auctor purus nec libero sodales, quis consectetur eros hendrerit. Vivamus varius semper neque, at tincidunt neque sollicitudin at.

### 5. 🧮 Error Propagation
   - Input: Variables, their values, and their uncertainties.
   - Output: Mean value, its uncertainty, and the result can be copied as LaTeX code for easy inclusion in documents.

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed dictum consectetur lectus, vitae tincidunt urna accumsan vel. Nunc in enim nulla. Vestibulum eget lorem nec nisi lobortis feugiat. Nullam lacinia pharetra lorem, vitae hendrerit dui fermentum nec. Sed non felis nec justo posuere interdum. Integer euismod magna nec dui vehicula, id fermentum orci hendrerit. Integer scelerisque nec urna vel lacinia. Vivamus convallis, dui a euismod lacinia, ex enim tincidunt magna, vitae suscipit lacus enim non nisi. Curabitur id justo justo. Nunc accumsan eget arcu at rhoncus. Etiam at sapien vel odio scelerisque fermentum. Sed tincidunt, justo a ultricies malesuada, sem justo consequat tellus, et ultricies elit arcu ac lorem. Ut auctor purus nec libero sodales, quis consectetur eros hendrerit. Vivamus varius semper neque, at tincidunt neque sollicitudin at.

### 6. 🔬 Show Molecule
   - Input: SMILES
   - Output: Molecular structure

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed dictum consectetur lectus, vitae tincidunt urna accumsan vel. Nunc in enim nulla. Vestibulum eget lorem nec nisi lobortis feugiat. Nullam lacinia pharetra lorem, vitae hendrerit dui fermentum nec. Sed non felis nec justo posuere interdum. Integer euismod magna nec dui vehicula, id fermentum orci hendrerit. Integer scelerisque nec urna vel lacinia. Vivamus convallis, dui a euismod lacinia, ex enim tincidunt magna, vitae suscipit lacus enim non nisi. Curabitur id justo justo. Nunc accumsan eget arcu at rhoncus. Etiam at sapien vel odio scelerisque fermentum. Sed tincidunt, justo a ultricies malesuada, sem justo consequat tellus, et ultricies elit arcu ac lorem. Ut auctor purus nec libero sodales, quis consectetur eros hendrerit. Vivamus varius semper neque, at tincidunt neque sollicitudin at.


import function


In [31]:
from project_ppchem_tools_kit.smiles_to_molar_mass import smiles_to_molar_mass
from project_ppchem_tools_kit.name_to_smiles import name_to_smiles
from project_ppchem_tools_kit.display_molecule import display_molecule
from project_ppchem_tools_kit.make_graph import make_graph
from project_ppchem_tools_kit.linear_regression import linear_regression
from project_ppchem_tools_kit.error_calculation_interface import error_calculation_interface
from project_ppchem_tools_kit.relative_to_assets import relative_to_assets
from project_ppchem_tools_kit.display_result import display_result
from project_ppchem_tools_kit.welcome_message import welcome_message

### 🖥️ Interface Variables

- **Entry Input:** Input entry widget for user data.
- **Selected Radio:** The currently selected radio button.
- **Window:** Main application window.
- **X Label:** Label for the x-axis in plots.
- **Y Label:** Label for the y-axis in plots.
- **Title:** Title of the plot.
- **Result Text:** Text widget for displaying results or messages.

In [32]:
entry_input = None
selected_radio = None
window = None
x_label = None
y_label = None
title = None
result_text = None

### ⚙️ Input Processing Function

#### 🔄 Process Input Function
This function `process_input` handles the processing of user input and executes corresponding actions based on the selected radio button. It updates the `result_text` variable accordingly.


##### 🧮 Processing Logic
The function checks the value of `selected_radio` to determine the type of input and performs the following actions:
- If the input type is conversion from chemical formula to SMILES it generates the corresponding data.
- If the input type is conversion from SMILES to molar mass, it calculates the molar mass.
- If the input type is plotting or linear regression, it invokes the respective plotting functions.
- If the input type is displaying a molecule, it renders the molecule image.
- If the input type is error calculation, it displays the error calculation interface.

In [33]:


def process_input(event=None):
    global selected_radio, x_label, y_label, title, result_text
    if selected_radio.get() == "1":
        smiles_molecule = name_to_smiles(entry_input.get().strip())
        if smiles_molecule is not None:
            result_text = f"The SMILES for the molecule '{entry_input.get().strip()}' is: {smiles_molecule}"
        else:
            result_text = f"The molecule '{entry_input.get().strip()}' was not found in the PubChem database."
    elif selected_radio.get() == "2":
        molar_mass = smiles_to_molar_mass(entry_input.get().strip())
        if molar_mass is not None:
            result_text = f"The molar mass of the molecule is: {molar_mass} g/mol"
        else:
            result_text = "Invalid SMILES or molecule not found."
    elif selected_radio.get() in ["3", "4"]:
        if selected_radio.get() == "3":
            make_graph(entry_input, x_label, y_label, title)
        else:
            linear_regression(entry_input, x_label, y_label, title)
        return 
    elif selected_radio.get() == "5":  
        display_molecule(entry_input)  
        return 
    elif selected_radio.get() == "6":
        error_calculation_interface()
    else:
        result_text = "Please select an input type."

    display_result(result_text)

### 📁 File Handling Functions

#### 📂 Browse Excel File Function
The function `browse_excel_file` allows users to browse and select an Excel file using a file dialog. Once a file is selected, its filepath is displayed in the input field.

#### 🗑️ Clear Input Function
The function `clear_input` clears the content of the input field in the GUI.


In [34]:
def browse_excel_file():
    global entry_input
    filepath = filedialog.askopenfilename(title="Select Excel File", filetypes=(("Excel files", "*.xlsx"), ("All files", "*.*")))
    if filepath:
        try:
            entry_input.delete(0, tk.END)  
            entry_input.insert(0, filepath)  
            print("Selected Excel file:", filepath)
            print("File path copied to clipboard.")
        except Exception as e:
            print("Error:", e)

def clear_input(): 
    """Clears the content of the input field in the GUI.
    """
    entry_input.delete(0, tk.END)

### 🖥️ GUI Overview
- **Window Initialization**: The GUI window is created using `tk.Tk()` function.
- **Window Configuration**:
  - Title: "Project Tools-Kit"
  - Default input type: 1
  - Dimensions: 1000x800 pixels
  - Background color: "#1B262C"
- **Canvas Setup**: A canvas widget covers the window with a background color of "#1B262C".
- **Text Creation**: Labels for "Choose Input Type" and "Input" are added to the canvas.
- **Input Entry Field**: An entry field for user input is provided with specific styling.
- **Browse Button**: Allows users to browse for an Excel file.
- **Process Button**: Initiates processing of the input.
- **Title Button**: Displays a welcome message.
- **Logo Display**: The EPFL logo is shown on the GUI window.

In [35]:
window = tk.Tk()  

x_axis_label = tk.StringVar()
y_axis_label = tk.StringVar()
graph_title = tk.StringVar()

window.title("Project Tools-Kit")
input_type = tk.IntVar() 
input_type.set(1)

window.geometry("1000x800")
window.configure(bg = "#1B262C")

canvas = Canvas(window, bg="#1B262C", height=800, width=1000, bd=0, highlightthickness=0, relief="ridge")
canvas.place(x=0, y=0)

canvas.create_text(12.0, 194.0, anchor="nw", text="Choose Input Type :", fill="#BBE1FA", font=("Times New Roman", 27, "bold"))

canvas.create_text(92.0, 378.0, anchor="nw",text="Input :", fill="#BBE1FA", font=("Times New Roman", 30, "bold"))
entry_input = Entry(bd=1, bg="#0F4C75", fg="#BBE1FA", highlightthickness=0, font=("Times New Roman", 25))
entry_input.place(x=272.5, y=368.0, width=605.0, height=74.0)

button_image_browse = PhotoImage(file=relative_to_assets("button_browse.png")) 
button_browse = Button(image=button_image_browse, borderwidth=0, highlightthickness=0, command=browse_excel_file, relief="flat")
button_browse.place(x=409.0, y=500.0, width=154.2904052734375, height=60.0)

button_image_process = PhotoImage(file=relative_to_assets("button_process.png"))
button_process = Button(image=button_image_process, borderwidth=0, highlightthickness=0, command=process_input, relief="flat")
button_process.place(x=394.5, y=620.0, width=182.0, height=73.0)

button_image_title = PhotoImage(file=relative_to_assets("button_title.png"))
button_title = Button(image=button_image_title, borderwidth=0, highlightthickness=0, command=welcome_message, relief="flat")
button_title.place(x=273.0, y=36.0, width=455.0, height=90.0)



logo_image = PhotoImage(file=relative_to_assets('EPFL_logo.png'))
logo_label = tk.Label(window, image=logo_image)
logo_label.place(x=10.0, y=10.0, width=150.0, height=60.0)


### 🗑️ Clear Input Function

The `clear_input` function is responsible for clearing the content of the input field in the GUI.


In [36]:
def clear_input(): 
    entry_input.delete(0, tk.END)

### 📻 Radio Selection Function

The `on_radio_select` function is triggered when a radio button is selected. It sets the value of the `selected_radio` variable and clears the input field.


In [37]:
def on_radio_select(value):
    selected_radio.set(value)
    clear_input()

In [38]:
def create_radio_button(x, y, text, value):  
    radio_button = tk.Radiobutton(canvas, text=text, variable=selected_radio, value=value,
                                  command=lambda: on_radio_select(value),
                                  font=("Times New Roman", 20), bg="#1B262C")
    canvas.create_window(x, y, anchor="nw", window=radio_button)

### 📻 Radio Button Creation Function

The `create_radio_button` function is responsible for creating radio buttons on the canvas at specified coordinates with the given text and value. It binds the `on_radio_select` function to the radio button's command, ensuring that the selected value is updated when the radio button is clicked.


In [39]:
radio_button_data = [(291, 194, "Molecule Name", "1"),(291, 252, "Excel Graph", "3"),(700, 194, "Linear Regression", "4"), (700, 252, "Show Molecule", "5"),(469, 252, "Error calculation", "6"),(469, 194, "Molecular weight", "2")]  
selected_radio = tk.StringVar(value="none")
for data in radio_button_data:
    create_radio_button(*data)

### 🕒 Clock Update Function

The `update_clock` function retrieves the current time and date and updates the respective labels (`clock_label` for time and `date_label` for date) on the GUI window. It uses the `datetime.now()` function to get the current time and date in the desired format ("%H:%M:%S" for time and "%Y-%m-%d" for date). The function is then scheduled to run every second using the `window.after()` method.


In [40]:
def update_clock():
    current_time = datetime.now().strftime('%H:%M:%S')
    current_date = datetime.now().strftime('%Y-%m-%d')
    clock_label.config(text=current_time)
    date_label.config(text=current_date)
    window.after(1000, update_clock)

date_label = tk.Label(window, text="", font=("Times New Roman", 20), bg="#1B262C", fg="#FFFFFF")
date_label.place(x=890, y=10)

clock_label = tk.Label(window, text="", font=("Times New Roman", 20), bg="#1B262C", fg="#FFFFFF")
clock_label.place(x=890, y=40)
update_clock()

### 🚪 Window Closing Function

The `on_closing` function is triggered when the user attempts to close the GUI window. It displays a confirmation message asking if the user is sure they want to quit. If the user confirms, the window is destroyed, effectively closing the application.


In [41]:
def on_closing():
    global window
    if messagebox.askokcancel("Quit", "Are you sure you want to quit?"):
        window.destroy()
window.protocol("WM_DELETE_WINDOW", on_closing)

''

### 🎯 Event Binding and Main Loop

The `window.bind("<Return>", process_input)` statement binds the `<Return>` key press event to the `process_input` function, allowing users to trigger input processing by pressing the Enter key.

The `window.mainloop()` function starts the main event loop, which waits for events (such as key presses, mouse clicks, etc.) and responds accordingly. This function must be called at the end of the GUI script to keep the window open and responsive.


In [42]:
window.bind("<Return>", process_input)

window.mainloop()